In [1]:
# import os
# os.chdir('..')
# os.getcwd()

In [2]:
import os 
import pandas as pd
import numpy as np
# from src.config import METADATA_PATH, SPLIT_PATH, NEGIBOX_PATH, CHEXPERT_PATH

%load_ext autoreload
%autoreload 2

In [3]:
os.chdir('../')
from config import METADATA_PATH, SPLIT_PATH, NEGIBOX_PATH, CHEXPERT_PATH, DATA_PATH

In [4]:
df_split = pd.read_csv(SPLIT_PATH)
df_metadata = pd.read_csv(METADATA_PATH)

In [5]:
# initialize view with a mapping from ViewPosition
VIEW_MAP = {
    'AP': 'frontal',
    'PA': 'frontal',
    'LATERAL': 'lateral',
    'LL': 'lateral',
    'LPO': 'other',
    'RAO': 'other',
    'RPO': 'other',
    'LAO': 'other',
    # the below are overwritten in some instances by manual review
    'AP AXIAL': 'other',
    'XTABLE LATERAL': 'other',
    'AP LLD': 'other',
    'PA LLD': 'other',
    'L5 S1': 'other',
    'SWIMMERS': 'other',
    'AP RLD': 'other',
    'PA RLD': 'other',
}
df_metadata['view'] = df_metadata['ViewPosition'].map(VIEW_MAP)

In [6]:
nb = pd.read_csv(NEGIBOX_PATH)
cx = pd.read_csv(CHEXPERT_PATH)

In [7]:
df_metadata[df_metadata['view'] == 'frontal'].merge(df_split, on='dicom_id')['split'].value_counts(dropna=False, normalize=True)

train       0.977964
test        0.013985
validate    0.008051
Name: split, dtype: float64

In [8]:
nb.columns

Index(['subject_id', 'study_id', 'Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
       'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'],
      dtype='object')

In [9]:
labels_cols = ['Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity',  'Pleural Effusion',
       'Pleural Other', 'Pneumonia', 'Pneumothorax']

In [10]:
nb.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,10000032,56699142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,10000764,57375967,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN


In [11]:
def is_finding(row: pd.core.series.Series):
    return 1.0 in row.values 

In [12]:
nb['findings'] = nb.apply(lambda row: is_finding(row[labels_cols]),axis =1)

In [13]:
number_of_findings = nb['findings'].sum()# /227827
number_of_no_findings = len(nb[nb['No Finding']==1.0])

In [14]:
metadata_cols = ['dicom_id', 'subject_id', 'study_id', 'view']
nb_cols = labels_cols + ['subject_id', 'study_id', 'findings']
split_cols = ['dicom_id', 'split']

df = df_metadata[metadata_cols]\
        .merge(nb[nb_cols], on=['subject_id', 'study_id'])\
        .drop(columns=['subject_id', 'study_id'])\
        .merge(df_split[split_cols], on='dicom_id')
df = df[df.view == 'frontal']




df[['split', 'findings','dicom_id']].groupby(['split', 'findings']).count()#.apply(lambda x: x*100/x.sum())
# ['dicom_id']

dicom_id
split    findings          
test     False          801
         True          2602
train    False        96694
         True        141268
validate False          813
         True          1146

In [15]:
f = 813 
t = 1146 
s = f + t
f/s, t/s ,s

(0.41500765696784075, 0.5849923430321593, 1959)

In [16]:
df[['dicom_id','split','findings']].head()

,dicom_id,split,findings
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,train,False
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,train,False
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,train,False
5,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,train,False
6,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,train,False


In [17]:
# df.to_csv('dicoms_with_labels_and_splits.csv')

In [18]:
metadata_cols = ['dicom_id', 'subject_id', 'study_id', 'view']
nb_cols = ['Pleural Effusion', 'subject_id', 'study_id']
split_cols = ['dicom_id', 'split']

df = df_metadata[metadata_cols]\
        .merge(nb[nb_cols], on=['subject_id', 'study_id'])\
        .drop(columns=['subject_id', 'study_id'])\
        .merge(df_split[split_cols], on='dicom_id')
df = df[df.view == 'frontal']


df['Pleural Effusion'] = df['Pleural Effusion'].fillna('na')
mapper = {
    -1   : 0,
    'na' : 0,
    0    : 0,
    1    : 1
}
df['Pleural Effusion_mapped'] = df['Pleural Effusion'].map(mapper)


df.groupby(['split', 'Pleural Effusion_mapped']).count()
# ['dicom_id']

dicom_id    view  Pleural Effusion
split    Pleural Effusion_mapped                                    
test     0                            2327    2327              2327
         1                            1076    1076              1076
train    0                          182839  182839            182839
         1                           55123   55123             55123
validate 0                            1461    1461              1461
         1                             498     498               498

In [19]:
train_data = df[df.split == 'train']
train_data['Pleural Effusion_mapped'].unique()

array([0, 1], dtype=int64)

In [20]:
# from src.data.loaders import create_data_folder_for_model

# for label in train_data['Pleural Effusion_mapped'].unique():
#     create_data_folder_for_model(list(train_data['dicom_id']), type='train', label=str(label))